In [1]:
!pip install pycaret


In [2]:
!pip install markupsafe==2.0.1

  Using cached MarkupSafe-2.0.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (30 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.5
    Uninstalling MarkupSafe-2.1.5:
      Successfully uninstalled MarkupSafe-2.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Li

In [3]:
!pip install jinja2

In [4]:
import pandas as pd
import jinja2
from pycaret.regression  import *
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#Load the ETH data
#from google.colab import files
#files.upload()

In [6]:
!pip install yfinance


In [13]:
#Read the data
asset = pd.read_csv('/content/GAZP_210101_240717.csv')
asset.rename(columns={'<TICKER>':'Ticker', '<PER>': 'Period', '<DATE>': 'Date', '<TIME>': 'Time', '<OPEN>': 'Open',
                   '<HIGH>': 'High', '<LOW>': 'Low', '<CLOSE>':'Close', '<VOL>': 'Volume'},
                  inplace = True)
#import yfinance as yf
#index_names = ['TSLA']
#start_data_string = '2019-01-01'
#end_data_string = '2022-12-20'
#asset = yf.download(index_names, start=start_data_string, end=end_data_string)
#Set the data as the index
asset['Spread'] = (asset['High']-asset['Low'])/asset['Open']*100
asset['Spread'] = asset['Spread'].astype(float)


asset = asset.set_index(pd.DatetimeIndex(asset['Date'].values))
#Show the data
asset


,Ticker,Period,Date,Time,Open,High,Low,Close,Volume,Spread
1970-01-01 00:00:00.020210105,GAZP,D,20210105,0,212.71,216.63,211.64,212.71,55195430,2.345917
1970-01-01 00:00:00.020210106,GAZP,D,20210106,0,213.20,215.38,210.22,215.32,48913740,2.420263
1970-01-01 00:00:00.020210107,GAZP,D,20210107,0,215.48,216.80,213.07,214.38,47897590,1.731019
1970-01-01 00:00:00.020210109,GAZP,D,20210109,0,217.14,226.24,217.00,225.49,100717800,4.255319
1970-01-01 00:00:00.020210112,GAZP,D,20210112,0,225.05,231.00,221.81,227.60,112506470,4.083537
...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:00:00.020240712,GAZP,D,20240712,0,117.61,122.70,117.60,121.75,72079520,4.336366
1970-01-01 00:00:00.020240713,GAZP,D,20240713,0,121.90,122.47,118.23,119.65,48194040,3.478261
1970-01-01 00:00:00.020240716,GAZP,D,20240716,0,120.00,120.99,118.52,119.28,40496850,2.058333
1970-01-01 00:00:00.020240717,GAZP,D,20240717,0,119.28,126.01,118.61,124.74,93643180,6.203890


In [14]:
#Make a copy of data set
df3 = asset.copy()
#Store the data in a feature data set call 'X' and add in the target column
X = np.array(df3[['Date','Open','Spread']])
#Store the Spread as an array in a variable called 'y'
y = df3['Spread'].values
#Split the data into 85% traning and 15% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 0, shuffle=False)


In [15]:
train_data = pd.DataFrame(x_train, columns=['Data', 'Open' , 'Spread'])
train_data['category_column'] = train_data['Spread'].astype('category').cat.codes
train_data.tail(7)

,Data,Open,Spread,category_column
741,20231229.0,159.95,1.350422,166
742,20231230.0,159.49,1.034548,72
743,20240104.0,159.73,1.158204,103
744,20240105.0,161.51,0.396260,1
745,20240106.0,161.22,0.762933,24
746,20240109.0,162.43,0.738780,20
747,20240110.0,163.01,0.791362,26


In [16]:
#Get the test data and transform it to the dataframe
test_data = pd.DataFrame(x_test, columns=['Data', 'Open' , 'Spread'])
test_data['category_column'] = test_data['Spread'].astype('category').cat.codes

test_data.tail(7)

,Data,Open,Spread,category_column
125,20240710.0,127.99,4.547230,115
126,20240711.0,124.00,5.846774,128
127,20240712.0,117.61,4.336366,114
128,20240713.0,121.90,3.478261,108
129,20240716.0,120.00,2.058333,85
130,20240717.0,119.28,6.203890,129
131,20240718.0,125.12,2.030051,83


In [17]:
#Initialising the setup
s = setup(data=train_data, target='Spread', session_id=123, use_gpu = True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

,Description,Value
0,Session id,123
1,Target,Spread
2,Target type,Regression
3,Original data shape,"(748, 4)"
4,Transformed data shape,"(748, 4)"
5,Transformed train set shape,"(523, 4)"
6,Transformed test set shape,"(225, 4)"
7,Numeric features,3
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of

**Mean Absolute Error (MAE)**
Метрика измеряет среднюю сумму абсолютной разницы между фактическим значением и прогнозируемым значением.

**MSE** : метрика, которая сообщает нам среднеквадратичную разницу между прогнозируемыми значениями и фактическими значениями в наборе данных. Чем ниже MSE, тем лучше модель соответствует набору данных.

**RMSE** : метрика, которая сообщает нам квадратный корень из средней квадратичной разницы между прогнозируемыми значениями и фактическими значениями в наборе данных. Чем ниже RMSE, тем лучше модель соответствует набору данных.

**R2** – очень важный показатель, который используется для оценки производительности модели машинного обучения на основе регрессии. Он произносится как R в квадрате и также известен как коэффициент детерминации. Суть его работы заключается в измерении количества отклонений в прогнозах, объясненных набором данных. Проще говоря, это разница между выборками в наборе данных и прогнозами, сделанными моделью. Если значение R2 равно 1, это означает, что модель идеальна, а если ее значение равно 0, это означает, что модель будет плохо работать с неизвестным набором данных.

Среднеквадратичная логарифмическая ошибка **(RMSLE)**
Это просто RMSE, рассчитанная в логарифмическом масштабе. Фактически, чтобы вычислить его, мы берем логарифм наших прогнозов и целевых значений и вычисляем среднеквадратичное отклонение между ними. Цели обычно неотрицательны, но могут быть равны 0, а логарифм 0 не определен. Вот почему константа обычно добавляется к прогнозам и целям перед применением логарифмической операции.

Средняя абсолютная ошибка в процентах (**MAPE**)
Предпочтение относительной ошибки также может быть выражено с помощью средней абсолютной ошибки процента, MAPE. Для каждого объекта абсолютная ошибка делится на целевое значение, что дает относительную ошибку. MAPE также можно рассматривать как взвешенную версию MAE.

In [18]:
#Train on multiple models and compare them
compare_models(sort='R2')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.0970,0.7548,0.4963,0.9772,0.0189,0.0127,0.2410
et,Extra Trees Regressor,0.0832,0.8605,0.4918,0.9753,0.0195,0.0074,0.2430
dt,Decision Tree Regressor,0.0981,0.7441,0.5442,0.9712,0.0219,0.0104,0.0240
rf,Random Forest Regressor,0.0803,1.0006,0.4914,0.9709,0.0198,0.0073,0.2720
ada,AdaBoost Regressor,0.3879,0.8721,0.7320,0.9527,0.1457,0.2074,0.1340
xgboost,Extreme Gradient Boosting,0.2020,3.0593,1.1473,0.8769,0.0471,0.0190,0.1030
lightgbm,Light Gradient Boosting Machine,0.4565,4.8795,1.7433,0.7088,0.1283,0.0749,0.0650
knn,K Neighbors Regressor,0.5749,6.6208,2.1015,0.6053,0.1615,0.1081,0.0360
br,Bayesian Ridge,1.2327,8.6429,2.5094,0.4335,0.3361,0.4828,0.0210
llar,Lasso Least Angle Regression,1.2415,8.6489,2.5124,0.4313,0.3374,0.4900,0.0210


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

GradientBoostingRegressor(random_state=123)

In [19]:
#Show the actual values
test_data['Spread']

0      1.046734
1      0.613798
2      1.095048
3      0.695971
4      1.108525
         ...   
127    4.336366
128    3.478261
129    2.058333
130    6.203890
131    2.030051
Name: Spread, Length: 132, dtype: float64

Huber Regressor — это тип надежного алгоритма регрессии, который используется для работы с выбросами в наборе данных.

В отличие от традиционных алгоритмов регрессии, таких как линейная регрессия, регрессор Хубера менее чувствителен к выбросам и не игнорирует их при моделировании данных.

Регрессор Хубера использует комбинацию линейных и квадратичных функций потерь для вычисления ошибок в наборе данных.

Функция потерь регрессора Хубера определяется таким образом, что она менее чувствительна к выбросам, в то же время штрафуя за большие ошибки больше, чем за мелкие.

Регрессор Хубера предпочтительнее других алгоритмов регрессии при работе с наборами данных, которые имеют выбросы или с высокой степенью изменчивости.

Таким образом, регрессор Хубера является идеальной моделью для наборов данных с выбросами и высокой изменчивостью. Он использует комбинацию линейных и квадратичных функций потерь для вычисления ошибок в наборе данных и менее чувствителен к выбросам.

In [20]:
#Create the model
model_huber = create_model('huber')
#Get the predictions
predict_model(model_huber, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.2425,4.7665,2.1832,-0.0349,0.4300,0.4966
1,1.5798,7.2744,2.6971,-0.1101,0.5048,0.5222
2,1.1291,4.6064,2.1462,-0.0540,0.4002,0.4194
3,1.6134,8.2867,2.8787,-0.1273,0.5011,0.5420
4,1.2291,7.6770,2.7707,-0.0303,0.4267,0.4452
5,2.1579,47.5924,6.8987,-0.0412,0.5752,0.6321
6,1.2703,7.6380,2.7637,-0.0414,0.4386,0.4380
7,1.5063,10.7459,3.2781,-0.0325,0.5011,0.5642
8,1.2162,3.2741,1.8095,-0.0189,0.4201,0.6082


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,2.151185
1,20240112.0,162.919998,7,2.151185
2,20240112.0,162.550003,42,2.151185
3,20240116.0,163.800003,12,2.151185
4,20240116.0,163.279999,43,2.151185
...,...,...,...,...
127,20240712.0,117.610001,114,2.151249
128,20240712.0,121.900002,108,2.151249
129,20240716.0,120.000000,85,2.151249
130,20240716.0,119.279999,129,2.151249


Регрессия с наименьшим углом (LARS) — это метод, используемый для линейной регрессии, цель которого — эффективно подогнать линейную модель к многомерному набору данных со многими переменными. Это алгоритм, который выполняет как выбор переменных, так и регуляризацию за один шаг. Ключевая идея LARS состоит в том, чтобы двигаться в направлении, равноугольном между предикторными переменными и текущим остатком. Этот подход приводит к набору кусочно-линейных решений и эффективен для большого числа переменных-предикторов. LARS можно использовать как для стандартной линейной регрессии, так и для регрессии Лассо. Это особенно полезно в случаях, когда количество переменных-предикторов превышает количество наблюдений в наборе данных, ситуация, называемая проблемой «большое p, маленькое n». В целом, LARS — это мощный и эффективный инструмент для линейной регрессии, который может легко обрабатывать большие многомерные наборы данных.

In [21]:
#Create the model
model_lar = create_model('lar')
#Get the predictions
predict_model(model_lar, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9424,1.7734,1.3317,0.6150,0.2698,0.4366
1,1.0658,2.5422,1.5944,0.6120,0.3230,0.4617
2,1.1473,2.5941,1.6106,0.4064,0.3246,0.4939
3,1.3082,4.0615,2.0153,0.4475,0.3295,0.5289
4,1.1438,4.3695,2.0903,0.4136,0.3419,0.4696
5,1.7573,38.5505,6.2089,0.1566,0.4044,0.5360
6,1.2569,4.2433,2.0599,0.4214,0.3461,0.5131
7,1.2434,5.6933,2.3861,0.4530,0.3343,0.5509
8,1.1133,1.6598,1.2883,0.4834,0.3426,0.5965


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.258141
1,20240112.0,162.919998,7,-0.034900
2,20240112.0,162.550003,42,0.296591
3,20240116.0,163.800003,12,0.015521
4,20240116.0,163.279999,43,0.308531
...,...,...,...,...
127,20240712.0,117.610001,114,0.867136
128,20240712.0,121.900002,108,0.823419
129,20240716.0,120.000000,85,0.599105
130,20240716.0,119.279999,129,1.015046


Пассивный агрессивный регрессор — это алгоритм машинного обучения, принадлежащий к семейству алгоритмов онлайн-обучения.


Впервые он был представлен Crammer et al. (2006) и с тех пор широко используется в различных областях, таких как классификация текста, распознавание речи и распознавание изображений.


Название «пассивно-агрессивный регрессор» относится к его агрессивному правилу обновления, когда он агрессивно настраивает параметры модели на основе наблюдаемых данных, сохраняя при этом пассивный подход, чтобы избежать переобучения.


В частности, в каждом учебном примере алгоритм прогнозирует выходные данные модели на основе текущих параметров модели, а затем сравнивает их с истинными выходными данными. Если прогноз неверен, параметры модели агрессивно обновляются, чтобы исправить ошибку. Если прогноз верен, параметры модели обновляются пассивным образом, чтобы избежать переобучения.


Таким образом, пассивно-агрессивный регрессор — это мощный и универсальный алгоритм машинного обучения, который можно применять к широкому кругу задач.

In [22]:
#Create the model
model_par = create_model('xgboost')
#Get the predictions
predict_model(model_par, data=test_data.drop('Spread', axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0650,0.1064,0.3262,0.9769,0.0272,0.0121
1,0.0537,0.0198,0.1407,0.9970,0.0142,0.0122
2,0.0463,0.0142,0.1192,0.9967,0.0141,0.0120
3,0.1124,0.2366,0.4864,0.9678,0.0362,0.0210
4,0.1735,1.2325,1.1102,0.8346,0.0463,0.0154
5,0.6187,18.3051,4.2784,0.5995,0.1291,0.0315
6,0.0868,0.2283,0.4779,0.9689,0.0235,0.0116
7,0.2901,3.3800,1.8385,0.6752,0.0657,0.0235
8,0.0229,0.0013,0.0367,0.9996,0.0085,0.0103


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.879916
1,20240112.0,162.919998,7,0.658852
2,20240112.0,162.550003,42,0.881877
3,20240116.0,163.800003,12,0.699539
4,20240116.0,163.279999,43,0.945651
...,...,...,...,...
127,20240712.0,117.610001,114,1.183659
128,20240712.0,121.900002,108,1.172541
129,20240716.0,120.000000,85,1.100598
130,20240716.0,119.279999,129,1.242098


In [23]:
#Create the model
model_par = create_model('par')
#Get the predictions
predict_model(model_par, data=test_data.drop('Spread', axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.2045,5.4231,2.3288,-0.1774,0.4662,0.3680
1,1.6952,8.7360,2.9557,-0.3332,0.5898,0.4299
2,1.7780,7.5144,2.7412,-0.7194,0.6987,0.5612
3,3.6834,16.6790,4.0840,-1.2691,0.8453,2.1956
4,4.1165,19.7570,4.4449,-1.6516,0.9073,2.3977
5,2.2481,46.2286,6.7992,-0.0114,0.5752,0.8639
6,2.1614,8.4959,2.9148,-0.1584,0.5814,1.1302
7,1.6171,12.6148,3.5517,-0.2121,0.6019,0.4089
8,1.2585,4.4129,2.1007,-0.3734,0.5090,0.4143


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,2.978108
1,20240112.0,162.919998,7,2.978108
2,20240112.0,162.550003,42,2.978108
3,20240116.0,163.800003,12,2.978108
4,20240116.0,163.279999,43,2.978109
...,...,...,...,...
127,20240712.0,117.610001,114,2.978197
128,20240712.0,121.900002,108,2.978197
129,20240716.0,120.000000,85,2.978197
130,20240716.0,119.279999,129,2.978197


Лассо-регрессия — это особый тип метода линейной регрессии, используемый в статистическом моделировании и машинном обучении.


Это также известно как регуляризация L1 из-за штрафа за норму L1, наложенного на коэффициенты регрессии во время оптимизации.


Штраф за норму L1 уменьшает некоторые коэффициенты регрессии до нуля, тем самым выполняя выбор признаков, что помогает снизить сложность модели и переоснащение.


Регрессия Лассо подходит, когда количество предикторов велико, и не все из них могут быть важны для прогнозирования переменной результата.


Он широко используется в таких приложениях, как финансы, генетика и маркетинг, для извлечения соответствующих функций из большого набора предикторов.


Таким образом, регрессия Лассо — это мощный метод, который выполняет выбор признаков в модели линейной регрессии. Он широко используется в различных областях для извлечения полезной информации из наборов данных с большим количеством предикторов.

In [24]:
#Create the model
model_lasso = create_model('lasso')
#Get the predictions
predict_model(model_lasso, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9312,1.7627,1.3277,0.6173,0.2656,0.4265
1,1.0515,2.5254,1.5891,0.6146,0.3184,0.4480
2,1.1275,2.5529,1.5978,0.4159,0.3246,0.4829
3,1.2834,4.0201,2.0050,0.4531,0.3264,0.5127
4,1.1337,4.3720,2.0909,0.4132,0.3391,0.4608
5,1.7318,38.6075,6.2135,0.1553,0.3991,0.5154
6,1.2466,4.2373,2.0585,0.4223,0.3455,0.5038
7,1.2296,5.7007,2.3876,0.4523,0.3318,0.5340
8,1.0785,1.5800,1.2570,0.5083,0.3383,0.5702


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.270390
1,20240112.0,162.919998,7,-0.018226
2,20240112.0,162.550003,42,0.308186
3,20240116.0,163.800003,12,0.031080
4,20240116.0,163.279999,43,0.319668
...,...,...,...,...
127,20240712.0,117.610001,114,0.884122
128,20240712.0,121.900002,108,0.839571
129,20240716.0,120.000000,85,0.619447
130,20240716.0,119.279999,129,1.029105


In [25]:
#Create the model
model_lasso = create_model('lasso')
#Get the predictions
predict_model(model_lasso, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9312,1.7627,1.3277,0.6173,0.2656,0.4265
1,1.0515,2.5254,1.5891,0.6146,0.3184,0.4480
2,1.1275,2.5529,1.5978,0.4159,0.3246,0.4829
3,1.2834,4.0201,2.0050,0.4531,0.3264,0.5127
4,1.1337,4.3720,2.0909,0.4132,0.3391,0.4608
5,1.7318,38.6075,6.2135,0.1553,0.3991,0.5154
6,1.2466,4.2373,2.0585,0.4223,0.3455,0.5038
7,1.2296,5.7007,2.3876,0.4523,0.3318,0.5340
8,1.0785,1.5800,1.2570,0.5083,0.3383,0.5702


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.270390
1,20240112.0,162.919998,7,-0.018226
2,20240112.0,162.550003,42,0.308186
3,20240116.0,163.800003,12,0.031080
4,20240116.0,163.279999,43,0.319668
...,...,...,...,...
127,20240712.0,117.610001,114,0.884122
128,20240712.0,121.900002,108,0.839571
129,20240716.0,120.000000,85,0.619447
130,20240716.0,119.279999,129,1.029105


In [26]:
#Create the model
model_lasso = create_model('lasso')
#Get the predictions
predict_model(model_lasso, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9312,1.7627,1.3277,0.6173,0.2656,0.4265
1,1.0515,2.5254,1.5891,0.6146,0.3184,0.4480
2,1.1275,2.5529,1.5978,0.4159,0.3246,0.4829
3,1.2834,4.0201,2.0050,0.4531,0.3264,0.5127
4,1.1337,4.3720,2.0909,0.4132,0.3391,0.4608
5,1.7318,38.6075,6.2135,0.1553,0.3991,0.5154
6,1.2466,4.2373,2.0585,0.4223,0.3455,0.5038
7,1.2296,5.7007,2.3876,0.4523,0.3318,0.5340
8,1.0785,1.5800,1.2570,0.5083,0.3383,0.5702


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.270390
1,20240112.0,162.919998,7,-0.018226
2,20240112.0,162.550003,42,0.308186
3,20240116.0,163.800003,12,0.031080
4,20240116.0,163.279999,43,0.319668
...,...,...,...,...
127,20240712.0,117.610001,114,0.884122
128,20240712.0,121.900002,108,0.839571
129,20240716.0,120.000000,85,0.619447
130,20240716.0,119.279999,129,1.029105


Регрессия байесовского хребта — это статистическая модель, используемая для регрессионного анализа, который представляет собой метод прогнозирования значений переменной результата на основе одной или нескольких переменных-предикторов.


Это байесовский метод, означающий, что он включает размещение априорных распределений в параметрах модели, которые затем обновляются с использованием данных для вычисления апостериорных распределений.


Часть названия Ridge происходит от того факта, что в нем используется штрафной член для размера коэффициентов, что помогает предотвратить переоснащение и повысить стабильность модели.


Этот тип регрессии может быть особенно полезен при работе с наборами данных с большим количеством предикторов или при высокой степени мультиколлинеарности между предикторами.


Регрессия байесовского хребта использовалась в широком спектре приложений, включая финансы, моделирование климата и медицинские исследования.


Таким образом, регрессия байесовского хребта — это мощный статистический метод, который может помочь исследователям и аналитикам построить точные прогностические модели даже в сложных наборах данных, где традиционные методы регрессии неэффективны.

In [27]:
#Create the model
model_br = create_model('br')
#Get the predictions
predict_model(model_br, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9244,1.7555,1.3250,0.6189,0.2629,0.4200
1,1.0454,2.5150,1.5859,0.6162,0.3167,0.4420
2,1.1178,2.5335,1.5917,0.4203,0.3252,0.4775
3,1.2712,4.0014,2.0003,0.4556,0.3248,0.5048
4,1.1246,4.3673,2.0898,0.4139,0.3380,0.4539
5,1.7260,38.6240,6.2148,0.1550,0.3977,0.5104
6,1.2402,4.2332,2.0575,0.4228,0.3437,0.4981
7,1.2210,5.7059,2.3887,0.4518,0.3314,0.5244
8,1.0614,1.5404,1.2411,0.5206,0.3372,0.5578


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.285595
1,20240112.0,162.919998,7,-0.000741
2,20240112.0,162.550003,42,0.323085
3,20240116.0,163.800003,12,0.048136
4,20240116.0,163.279999,43,0.334446
...,...,...,...,...
127,20240712.0,117.610001,114,0.895932
128,20240712.0,121.900002,108,0.851570
129,20240716.0,120.000000,85,0.633271
130,20240716.0,119.279999,129,1.039694


Эластичная чистая регрессия — это статистический метод, используемый для регрессионного анализа. Это комбинация методов регрессии Лассо и Риджа. Метод эластичной сети используется для работы с мультиколлинеарностью, когда независимые переменные сильно коррелируют друг с другом. Регрессия эластичной сети применяет штраф к коэффициентам в модели, который контролирует величину усадки, применяемую к каждому параметру. Этот метод полезен в ситуациях с большим количеством независимых переменных и ограниченными данными. Регрессия эластичной сети приводит к подмножеству наиболее важных переменных, что делает модель более интерпретируемой. Это также помогает предотвратить переоснащение и повышает точность прогнозов. По сравнению с другими методами регрессии регрессия Elastic Net уравновешивает сложность модели и эффективность прогнозирования и является полезным инструментом современного анализа данных. В целом регрессия эластичной сети представляет собой универсальный статистический метод, который служит полезным инструментом для прогнозирования взаимосвязей между переменными.

In [28]:
#Create the model
model_en = create_model('en')
#Get the predictions
predict_model(model_en, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9308,1.7616,1.3273,0.6175,0.2654,0.4264
1,1.0526,2.5329,1.5915,0.6135,0.3189,0.4485
2,1.1272,2.5586,1.5995,0.4146,0.3242,0.4819
3,1.2852,4.0268,2.0067,0.4522,0.3269,0.5137
4,1.1323,4.3663,2.0896,0.4140,0.3395,0.4605
5,1.7347,38.6029,6.2131,0.1554,0.3995,0.5175
6,1.2468,4.2352,2.0580,0.4225,0.3449,0.5043
7,1.2291,5.6993,2.3873,0.4524,0.3322,0.5352
8,1.0832,1.5943,1.2626,0.5039,0.3388,0.5729


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.280822
1,20240112.0,162.919998,7,-0.007696
2,20240112.0,162.550003,42,0.318655
3,20240116.0,163.800003,12,0.041833
4,20240116.0,163.279999,43,0.330321
...,...,...,...,...
127,20240712.0,117.610001,114,0.884529
128,20240712.0,121.900002,108,0.841017
129,20240716.0,120.000000,85,0.620416
130,20240716.0,119.279999,129,1.029936


Ридж-регрессия — это модель линейной регрессии, используемая для решения проблемы мультиколлинеарности данных. Он добавляет штрафной член к целевой функции наименьших квадратов, что уменьшает веса переменных в модели за счет добавления члена регуляризации к функции стоимости. Этот член регуляризации предотвращает переоснащение модели обучающими данными за счет снижения значений коэффициентов регрессии.
Ридж-регрессия — это мощный метод обработки многомерных наборов данных с коррелированными функциями. Это может помочь создать более надежные и надежные прогностические модели за счет уменьшения влияния мультиколлинеарности. Ридж-регрессия широко используется в различных областях, таких как финансы, экономика, здравоохранение и инженерия.
Чтобы выполнить ридж-регрессию, нужно сначала стандартизировать переменные-предикторы, чтобы они имели нулевое среднее значение и единичную дисперсию. Затем модель обучается с помощью параметра настройки, называемого лямбда, посредством перекрестной проверки. Значение лямбда определяется балансом между сложностью модели и производительностью. В целом, ридж-регрессия является ценным инструментом для прогнозного моделирования и может

In [29]:
#Create the model
model_ridge = create_model('ridge')
#Get the predictions
predict_model(model_ridge, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9310,1.7609,1.3270,0.6177,0.2653,0.4265
1,1.0537,2.5406,1.5939,0.6123,0.3195,0.4489
2,1.1270,2.5646,1.6014,0.4132,0.3237,0.4809
3,1.2870,4.0338,2.0084,0.4512,0.3273,0.5147
4,1.1315,4.3609,2.0883,0.4147,0.3399,0.4606
5,1.7378,38.5985,6.2128,0.1555,0.4001,0.5197
6,1.2470,4.2332,2.0575,0.4228,0.3443,0.5048
7,1.2286,5.6982,2.3871,0.4525,0.3326,0.5364
8,1.0880,1.6091,1.2685,0.4992,0.3394,0.5756


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.291147
1,20240112.0,162.919998,7,0.002723
2,20240112.0,162.550003,42,0.329019
3,20240116.0,163.800003,12,0.052477
4,20240116.0,163.279999,43,0.340869
...,...,...,...,...
127,20240712.0,117.610001,114,0.884821
128,20240712.0,121.900002,108,0.842348
129,20240716.0,120.000000,85,0.621267
130,20240716.0,119.279999,129,1.030656


Orthogonal Matching Pursuit (OMP) — это широко используемый алгоритм для решения задач разреженной аппроксимации, целью которого является представление сигнала или точки данных в виде суммы нескольких выбранных атомов из заданного словаря. Алгоритм работает путем итеративного выбора из словаря атомов, которые имеют наибольшую корреляцию с остаточным сигналом, а затем соответствующим образом обновляет остаточный сигнал и набор выбора.
OMP имеет множество применений в обработке сигналов, распознавании образов, машинном обучении и вычислительной визуализации. Он известен своей простотой, эффективностью и надежностью и часто используется в качестве эталона для других алгоритмов разреженной аппроксимации.
Одним из основных преимуществ OMP является то, что он обеспечивает доказуемую гарантию ошибки аппроксимации, которая связана с разреженностью основного сигнала и когерентностью словаря. Это делает OMP надежным инструментом для решения многих практических задач, когда уровень разреженности и словарь могут быть неизвестны или меняться со временем.
В целом, OMP предлагает мощный и универсальный подход к разреженным

In [30]:
#Create the model
model_omp = create_model('omp')
#Get the predictions
predict_model(model_omp, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.5327,4.7232,2.1733,-0.0255,0.4711,0.7839
1,1.7351,6.4755,2.5447,0.0118,0.4958,0.7373
2,1.3543,4.4463,2.1086,-0.0174,0.4291,0.6588
3,1.5701,7.2894,2.6999,0.0083,0.4750,0.6793
4,1.5546,7.5999,2.7568,-0.0200,0.4715,0.7332
5,2.2270,46.1795,6.7956,-0.0103,0.5711,0.8455
6,1.4799,7.2734,2.6969,0.0083,0.4588,0.6532
7,1.8330,10.3820,3.2221,0.0025,0.5340,0.8718
8,1.3792,3.3540,1.8314,-0.0438,0.4697,0.8714


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,2.366471
1,20240112.0,162.919998,7,2.366471
2,20240112.0,162.550003,42,2.366471
3,20240116.0,163.800003,12,2.366371
4,20240116.0,163.279999,43,2.366371
...,...,...,...,...
127,20240712.0,117.610001,114,2.351497
128,20240712.0,121.900002,108,2.351497
129,20240716.0,120.000000,85,2.351397
130,20240716.0,119.279999,129,2.351397


Линейная регрессия — это статистический подход, используемый для установления взаимосвязи между зависимой переменной и одной или несколькими независимыми переменными. Цель линейной регрессии состоит в том, чтобы найти наиболее подходящую линию, которая предсказывает зависимую переменную на основе значений независимой переменной (переменных). Этот метод обычно используется в различных областях, таких как экономика, финансы, наука и техника, для прогнозирования и выявления тенденций.
Чтобы выполнить линейную регрессию, входные данные должны быть сначала нанесены на точечный график. Оттуда по математической формуле рассчитывается линия наилучшего соответствия. Наиболее распространенным методом, используемым для нахождения этой линии, является метод обычных наименьших квадратов (OLS).
Линейная регрессия имеет ряд преимуществ, таких как простота и удобство использования. Это также мощный инструмент для обнаружения взаимосвязей и закономерностей в данных. Однако важно отметить, что линейная регрессия предполагает линейную зависимость между переменными, которая не всегда может быть точной. Он также чувствителен к выбросам, которые могут существенно повлиять на точность модели.


In [31]:
#Create the model
model_lr = create_model('lr')
#Get the predictions
predict_model(model_lr, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.9310,1.7609,1.3270,0.6177,0.2653,0.4265
1,1.0537,2.5406,1.5939,0.6123,0.3195,0.4489
2,1.1270,2.5646,1.6014,0.4132,0.3237,0.4809
3,1.2870,4.0338,2.0084,0.4512,0.3273,0.5147
4,1.1315,4.3609,2.0883,0.4147,0.3399,0.4606
5,1.7378,38.5985,6.2128,0.1555,0.4001,0.5197
6,1.2470,4.2332,2.0575,0.4228,0.3443,0.5048
7,1.2286,5.6982,2.3871,0.4525,0.3326,0.5364
8,1.0880,1.6091,1.2685,0.4992,0.3394,0.5756


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.291147
1,20240112.0,162.919998,7,0.002723
2,20240112.0,162.550003,42,0.329019
3,20240116.0,163.800003,12,0.052477
4,20240116.0,163.279999,43,0.340869
...,...,...,...,...
127,20240712.0,117.610001,114,0.884821
128,20240712.0,121.900002,108,0.842348
129,20240716.0,120.000000,85,0.621267
130,20240716.0,119.279999,129,1.030656


Gradient Boosting Regressor — это мощный алгоритм машинного обучения, используемый для регрессионного анализа.


Это тип алгоритма повышения, в котором слабый ученик строится и итеративно подстраивается под данные.


Целью Gradient Boosting Regressor является минимизация функции потерь модели путем корректировки весов и предикторов слабого ученика на каждой итерации.


Термин «Градиент» относится к направлению алгоритма оптимизации градиентного спуска, который используется для обновления параметров слабых учеников.


Gradient Boosting Regressor очень эффективен для сложных нелинейных задач, поскольку он может обрабатывать множество функций и взаимодействий между ними.


Он широко используется в таких областях, как финансы, маркетинг и здравоохранение, для прогнозирования результатов и принятия решений на основе прогнозов.


Хотя регрессор с усилением градиента может обеспечить точные прогнозы, он также может быть подвержен переоснащению и требует тщательной настройки своих гиперпараметров.

In [32]:
#Create the model
model_gbr = create_model('gbr')
#Get the predictions
predict_model(model_gbr, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0501,0.0254,0.1592,0.9945,0.0144,0.0117
1,0.0568,0.0311,0.1763,0.9953,0.0152,0.0121
2,0.0399,0.0124,0.1112,0.9972,0.0115,0.0107
3,0.0471,0.0149,0.1220,0.9980,0.0123,0.0117
4,0.0370,0.0145,0.1202,0.9981,0.0097,0.0102
5,0.3202,4.3897,2.0952,0.9040,0.0498,0.0214
6,0.0367,0.0132,0.1147,0.9982,0.0098,0.0090
7,0.0592,0.0604,0.2457,0.9942,0.0141,0.0116
8,0.0340,0.0085,0.0922,0.9974,0.0108,0.0101


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.874560
1,20240112.0,162.919998,7,0.643355
2,20240112.0,162.550003,42,0.893960
3,20240116.0,163.800003,12,0.685930
4,20240116.0,163.279999,43,0.897636
...,...,...,...,...
127,20240712.0,117.610001,114,1.186863
128,20240712.0,121.900002,108,1.185180
129,20240716.0,120.000000,85,1.084935
130,20240716.0,119.279999,129,1.211633


Extra Trees Regressor — это алгоритм машинного обучения, используемый для задач регрессии.
Он принадлежит к семейству ансамблевых алгоритмов, которые объединяют несколько моделей для повышения общей производительности.
Он работает путем случайного выбора подмножеств функций и выборок данных для создания деревьев решений.
Окончательный прогноз определяется путем агрегирования прогнозов отдельных деревьев решений.
Алгоритм Extra Trees Regressor известен своей способностью обрабатывать многомерные наборы данных и зашумленные данные.
Он также имеет то преимущество, что является вычислительно эффективным и может быть легко распараллелен.
Некоторые из недостатков этого алгоритма включают возможность переобучения, если количество деревьев решений слишком велико, и невозможность обрабатывать нелинейные отношения между функциями.
В общем, Extra Trees Regressor может быть полезным инструментом для задач регрессии, когда данные являются многомерными и зашумленными, но его следует использовать в сочетании с другими алгоритмами для обеспечения наилучших результатов.


In [33]:
#Create the model
model_et = create_model('et')
#Get the predictions
predict_model(model_et, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0389,0.0381,0.1952,0.9917,0.0146,0.0060
1,0.0245,0.0066,0.0812,0.9990,0.0067,0.0043
2,0.0136,0.0011,0.0339,0.9997,0.0050,0.0034
3,0.0409,0.0207,0.1438,0.9972,0.0105,0.0065
4,0.0620,0.1427,0.3778,0.9808,0.0177,0.0052
5,0.4114,7.9256,2.8153,0.8266,0.0724,0.0221
6,0.0628,0.1516,0.3894,0.9793,0.0188,0.0057
7,0.0602,0.1152,0.3394,0.9889,0.0152,0.0061
8,0.0223,0.0107,0.1035,0.9967,0.0099,0.0049


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.866214
1,20240112.0,162.919998,7,0.600103
2,20240112.0,162.550003,42,0.883677
3,20240116.0,163.800003,12,0.660915
4,20240116.0,163.279999,43,0.891590
...,...,...,...,...
127,20240712.0,117.610001,114,1.167366
128,20240712.0,121.900002,108,1.154416
129,20240716.0,120.000000,85,1.079592
130,20240716.0,119.279999,129,1.210993


Random Forest Regressor — это алгоритм машинного обучения, используемый для задач регрессии.
Он состоит из ряда деревьев решений (леса), которые генерируются случайным образом и затем используются для прогнозирования путем усреднения результатов отдельных деревьев.
Этот метод помогает повысить точность прогнозов по сравнению с одним деревом решений.
Random Forest Regressor используется в различных областях, включая финансы, медицину и розничную торговлю, для прогнозирования тенденций и принятия бизнес-решений.
Он хорошо подходит для обработки больших и сложных наборов данных, а также может в некоторой степени обрабатывать отсутствующие значения и выбросы данных.
Алгоритм прост в использовании и реализации, что делает его популярным среди энтузиастов машинного обучения и новичков.
Однако основным недостатком Random Forest Regressor является то, что он может требовать значительных вычислительных ресурсов и времени, особенно для больших наборов данных.
Таким образом, Random Forest Regressor — это эффективный алгоритм машинного обучения, который широко используется.

In [34]:
#Create the model
model_rf = create_model('rf')
#Get the predictions
predict_model(model_rf, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0711,0.1873,0.4328,0.9593,0.0289,0.0082
1,0.0217,0.0044,0.0661,0.9993,0.0063,0.0047
2,0.0147,0.0009,0.0299,0.9998,0.0053,0.0044
3,0.0362,0.0279,0.1670,0.9962,0.0096,0.0050
4,0.0655,0.1662,0.4077,0.9777,0.0188,0.0062
5,0.4421,9.4565,3.0751,0.7931,0.0800,0.0215
6,0.0241,0.0082,0.0904,0.9989,0.0081,0.0040
7,0.0542,0.0911,0.3018,0.9913,0.0148,0.0057
8,0.0249,0.0157,0.1251,0.9951,0.0114,0.0052


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.871786
1,20240112.0,162.919998,7,0.627023
2,20240112.0,162.550003,42,0.888612
3,20240116.0,163.800003,12,0.670807
4,20240116.0,163.279999,43,0.890272
...,...,...,...,...
127,20240712.0,117.610001,114,1.186815
128,20240712.0,121.900002,108,1.166176
129,20240716.0,120.000000,85,1.094163
130,20240716.0,119.279999,129,1.224170


K Neighbours Regressor — это алгоритм машинного обучения, используемый для задач регрессии.
Он работает, предсказывая вывод точки данных на основе значений ее k ближайших соседей.
Алгоритм вычисляет расстояния между точками данных и выбирает k ближайших соседей каждой точки.
Затем требуется среднее значение выходных данных k соседей, чтобы предсказать выходные данные рассматриваемой точки данных.
K Neighbours Regressor — это непараметрический алгоритм, то есть он не делает никаких предположений о функциональной форме взаимосвязи между признаками и выходными данными.
Он часто используется, когда нет четкой линейной зависимости между входными и выходными переменными, и особенно полезен для прогнозирования непрерывных числовых значений.
Одним из недостатков алгоритма является то, что он может быть чувствителен к масштабу входных признаков, поэтому может потребоваться предварительная обработка для обеспечения точных прогнозов.

In [35]:
#Create the model
model_knn = create_model('knn')
#Get the predictions
predict_model(model_knn, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3506,0.9923,0.9961,0.7846,0.1245,0.0998
1,0.4811,1.0519,1.0256,0.8395,0.1236,0.1035
2,0.5088,2.4303,1.5589,0.4439,0.1575,0.1153
3,0.7119,4.0294,2.0073,0.4518,0.1915,0.1291
4,0.4759,3.6900,1.9209,0.5048,0.1677,0.0939
5,1.1518,30.8974,5.5585,0.3240,0.2394,0.1311
6,0.4342,1.9921,1.4114,0.7284,0.1374,0.0835
7,0.4848,2.5594,1.5998,0.7541,0.1259,0.1010
8,0.3416,0.4641,0.6812,0.8556,0.1079,0.1010


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.758877
1,20240112.0,162.919998,7,0.758877
2,20240112.0,162.550003,42,0.758877
3,20240116.0,163.800003,12,0.758877
4,20240116.0,163.279999,43,0.758877
...,...,...,...,...
127,20240712.0,117.610001,114,0.901272
128,20240712.0,121.900002,108,0.901272
129,20240716.0,120.000000,85,0.758877
130,20240716.0,119.279999,129,0.901272


AdaBoost Regressor — популярный алгоритм машинного обучения, используемый для задач регрессии.


Он принадлежит к семейству алгоритмов повышения, которые работают путем объединения нескольких более слабых моделей для формирования более сильной общей модели.


AdaBoost Regressor работает путем итеративного обучения моделей для исправления ошибок предыдущей модели.


Алгоритм присваивает больший вес точкам данных, которые были неправильно классифицированы в предыдущей итерации, заставляя последующие модели сосредоточиться на улучшении этих экземпляров.


AdaBoost Regressor является гибким и может работать с различными базовыми моделями регрессии, включая деревья решений, линейные модели и регрессию опорных векторов.


Однако он чувствителен к выбросам и зашумленным точкам данных, которые могут повлиять на точность окончательной модели.


AdaBoost Regressor — это популярный алгоритм, используемый в таких приложениях, как прогнозирование цен на акции, анализ поведения клиентов и оптимизация маркетинговых кампаний.

In [36]:
#Create the model
model_ada = create_model('ada')
#Get the predictions
ada =predict_model(model_ada, data=test_data.drop(['Spread'], axis=1))
ada

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.3305,0.1604,0.4005,0.9652,0.1459,0.2074
1,0.4240,0.2665,0.5162,0.9593,0.1720,0.2497
2,0.3022,0.1277,0.3574,0.9708,0.1230,0.1592
3,0.3372,0.1690,0.4111,0.9770,0.1305,0.1857
4,0.3517,0.1847,0.4297,0.9752,0.1341,0.1806
5,0.5179,3.5638,1.8878,0.9220,0.1486,0.2164
6,0.3639,0.2077,0.4558,0.9717,0.1289,0.1756
7,0.3817,0.2736,0.5231,0.9737,0.1563,0.2297
8,0.3613,0.2031,0.4507,0.9368,0.1766,0.2702


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,1.400175
1,20240112.0,162.919998,7,1.400175
2,20240112.0,162.550003,42,1.400175
3,20240116.0,163.800003,12,1.400175
4,20240116.0,163.279999,43,1.400175
...,...,...,...,...
127,20240712.0,117.610001,114,1.400175
128,20240712.0,121.900002,108,1.400175
129,20240716.0,120.000000,85,1.400175
130,20240716.0,119.279999,129,1.400175


In [37]:
last_ada_prediction = ada.iloc[-1]
last_ada_prediction

Data                2.024072e+07
Open                1.251200e+02
category_column     8.300000e+01
prediction_label    1.400175e+00
Name: 131, dtype: float64

Регрессор дерева решений — это алгоритм машинного обучения, используемый для задач регрессии.
Он работает путем рекурсивного разделения данных на основе функции, которая обеспечивает наибольшую информационную выгоду.
Конечным результатом является древовидная структура, которую можно использовать для прогнозирования новых данных.
Регрессоры деревьев решений популярны, потому что их легко понять и интерпретировать, и они могут обрабатывать как категориальные, так и числовые данные.
Однако они могут быть склонны к переоснащению, которое можно облегчить с помощью таких методов, как перекрестная проверка или обрезка.
Регрессоры деревьев решений обычно используются в различных областях, таких как финансы, медицина и инженерия, для таких задач, как прогнозирование продаж, диагностика заболеваний и определение важных функций в системе.

In [38]:
#Create the model
model_dt = create_model('dt')
#Get the predictions
predict_model(model_dt, data=test_data.drop(['Spread'], axis=1))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0397,0.0241,0.1553,0.9948,0.0126,0.0074
1,0.0550,0.0312,0.1765,0.9952,0.0150,0.0106
2,0.0835,0.2148,0.4635,0.9508,0.0283,0.0112
3,0.0442,0.0147,0.1214,0.9980,0.0114,0.0094
4,0.0287,0.0150,0.1225,0.9980,0.0075,0.0060
5,0.2816,3.4526,1.8581,0.9245,0.0430,0.0151
6,0.0770,0.1576,0.3970,0.9785,0.0364,0.0110
7,0.0591,0.0630,0.2509,0.9940,0.0152,0.0099
8,0.0163,0.0012,0.0346,0.9996,0.0058,0.0057


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Data,Open,category_column,prediction_label
0,20240112.0,162.410004,38,0.868443
1,20240112.0,162.919998,7,0.640683
2,20240112.0,162.550003,42,0.892319
3,20240116.0,163.800003,12,0.684728
4,20240116.0,163.279999,43,0.892319
...,...,...,...,...
127,20240712.0,117.610001,114,1.186559
128,20240712.0,121.900002,108,1.171806
129,20240716.0,120.000000,85,1.092006
130,20240716.0,119.279999,129,1.231066


In [39]:
pred_0_Spread = (5.32+3.82+5.13+7.72+6.53+5.8+6.53+5.21+5.85+5.13+5.35+1.92+4.91+7.42+7.21+3.1)/16
real_Spread_0 = 2.28
mistake_0 = (pred_0_Spread-real_Spread_0)
pred_0_Spread, real_Spread_0, mistake_0

(5.434374999999999, 2.28, 3.1543749999999995)

In [40]:
pred_10_Spread = (5.32+3.82+5.13+7.72+6.53+5.8+6.53+5.21+5.85+5.13+5.35+1.92+4.91+7.42+7.21+3.1)/16
real_Spread_10 = 2.28
mistake_10 = (pred_10_Spread-real_Spread_10)
pred_10_Spread, real_Spread_10, mistake_10

(5.434374999999999, 2.28, 3.1543749999999995)

In [41]:
pred_18_Spread = (5.32+3.82+5.13+7.72+6.53+5.8+6.53+5.21+5.85+5.13+5.35+1.92+4.91+7.42+7.21+3.1)/16
real_Spread_18 = 1.68
mistake_18 = (pred_18_Spread-real_Spread_18)
pred_18_Spread, real_Spread_18, mistake_18

(5.434374999999999, 1.68, 3.7543749999999996)

In [42]:
quote = 344000
spread = quote*0.3/100
buy = quote - spread/2
sell = quote + spread/2
spread,buy,sell

(1032.0, 343484.0, 344516.0)